<a href="https://colab.research.google.com/github/jackschedel/AutoCalibr/blob/main/AutoCalibr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# mount Google Drive to VM disk
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# test load single .fbx file
loadedFile = open('/content/drive/MyDrive/datasets/AutoCalibr/fbx/Beloved.fbx', 'rb')

data = loadedFile.read()

In [4]:
print("Length of this single example file: ", len(data))

Length of this single example file:  822892
